In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Define model architecture
def create_asl_image_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Path to train directory
train_dir = 'asl_alphabet_train\asl_alphabet_train'

In [ ]:
# Data generators for training data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(64, 64), batch_size=32, class_mode='sparse')

In [ ]:
# Create and train the model
input_shape = (64, 64, 3)
num_classes = len(os.listdir(train_dir)) 
asl_image_model = create_asl_image_model(input_shape, num_classes)

In [ ]:
checkpoint = ModelCheckpoint(
    'sign_language.model.h5',
    monitor='accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

Early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=8,
    verbose=1,
    restore_best_weights=True
)


In [ ]:
# Train the model
asl_image_model.fit(train_generator, epochs=10, callbacks=[checkpoint,Early_stop])

In [ ]:
# Path to test directory
test_dir = 'asl_alphabet_test\asl_alphabet_test'

In [ ]:
# Function to load and preprocess an image
def load_and_preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((64, 64))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
# Create a dictionary to map class indices to class names
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

In [ ]:
# Predict on test images
test_images = [f for f in os.listdir(test_dir) if f.endswith(('jpg', 'jpeg', 'png'))]
predictions = []

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    img = load_and_preprocess_image(img_path)
    prediction = asl_image_model.predict(img)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = class_labels[predicted_class_index]
    predictions.append((img_name, predicted_class_label))

In [ ]:
# Display predictions
for img_name, predicted_class_label in predictions:
    print(f"Image: {img_name}, Predicted Class: {predicted_class_label}")